In [1]:
!pip install imbalanced-learn
!pip install --upgrade numexpr

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC

# Load the dataset
file_path = 'End_dataframe.csv'  # Update with your file path
data = pd.read_csv(file_path)

# Features and target
X = data[['Gender', 'Prior_Donation', 'Lcheek_max', 'Rcheek_max', 'nose_max', 'chin_max', 'below_nose_max', 'HRV_minmax']]
y = data['VVR_Encoded']  # Assuming 'VVR_Encoded' is the target variable

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to balance the classes in the training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Define the SVM model and parameter grid for Grid Search
svm = SVC(probability=True, random_state=42)
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [3, 4, 5],  
    'gamma': ['scale', 'auto']
}

# Grid Search with cross-validation
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=0)
grid_search.fit(X_train_resampled.drop(columns='Gender'), y_train_resampled)

# Best model from Grid Search
best_svm = grid_search.best_estimator_

# Make predictions on the test set and training set
y_pred_test = best_svm.predict(X_test.drop(columns='Gender'))
y_pred_train = best_svm.predict(X_train_resampled.drop(columns='Gender'))

# Balanced accuracy and per-class balanced accuracy
def per_class_balanced_accuracy(cm):
    per_class = cm.diagonal() / cm.sum(axis=1)
    return per_class

# Test set performance
test_cm = confusion_matrix(y_test, y_pred_test)
test_balanced_acc = balanced_accuracy_score(y_test, y_pred_test)
test_per_class_balanced_acc = per_class_balanced_accuracy(test_cm)

print("\nTest Set Performance:")
print(f"Balanced Accuracy (Test Set): {test_balanced_acc:.2f}")
for i, acc in enumerate(test_per_class_balanced_acc):
    print(f"Class {i} Balanced Accuracy: {acc:.2f}")
print("\nConfusion Matrix (Test Set):")
print(test_cm)
print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_pred_test))

# Training set performance
train_cm = confusion_matrix(y_train_resampled, y_pred_train)
train_balanced_acc = balanced_accuracy_score(y_train_resampled, y_pred_train)
train_per_class_balanced_acc = per_class_balanced_accuracy(train_cm)

print("\nTraining Set Performance:")
print(f"Balanced Accuracy (Training Set): {train_balanced_acc:.2f}")
for i, acc in enumerate(train_per_class_balanced_acc):
    print(f"Class {i} Balanced Accuracy: {acc:.2f}")
print("\nConfusion Matrix (Training Set):")
print(train_cm)
print("\nClassification Report (Training Set):")
print(classification_report(y_train_resampled, y_pred_train))



Test Set Performance:
Balanced Accuracy (Test Set): 0.50
Class 0 Balanced Accuracy: 0.30
Class 1 Balanced Accuracy: 0.70

Confusion Matrix (Test Set):
[[28 64]
 [11 26]]

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.72      0.30      0.43        92
           1       0.29      0.70      0.41        37

    accuracy                           0.42       129
   macro avg       0.50      0.50      0.42       129
weighted avg       0.59      0.42      0.42       129


Training Set Performance:
Balanced Accuracy (Training Set): 0.65
Class 0 Balanced Accuracy: 0.37
Class 1 Balanced Accuracy: 0.93

Confusion Matrix (Training Set):
[[133 231]
 [ 26 338]]

Classification Report (Training Set):
              precision    recall  f1-score   support

           0       0.84      0.37      0.51       364
           1       0.59      0.93      0.72       364

    accuracy                           0.65       728
   macro avg       0.72